<a href="https://colab.research.google.com/github/yang0256/Twitter_ETL/blob/main/tweeter_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# tweets collector
import requests
# from datetime import datetime, timedelta
# import os
# import json
import pandas as pd

# To set your environment variables in your terminal run the following line:


# Set the Bearer Token
bearer_token = "AAAAAAAAAAAAAAAAAAAAAN%2FhlQEAAAAAA3KWaOM4ynnWjSNqd%2BH4ncV6mvU%3DRG5HSsW7lgy9kJakzkn67aBPzmfnAHCELtudRJisaeeNUHlkoj"
# headers = {"Authorization": f"Bearer {bearer_token}"}
# 

search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
# query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}
# query_params = {'query': '(from:business -is:retweet) OR #business','tweet.fields': 'author_id'}
query_params = {'query': '(from:OttawaPolice -is:retweet) OR #OttawaPolice','tweet.fields': 'author_id'}

#lavienrosye michaeljburry

# query_params = {"query": "from:OttawaPolice", "max_results": 100, "tweet.fields": "public_metrics",  
                # "start_time":'2023-01-01T00:00:00.000Z', "end_time":'2023-02-02T12:00:00.000Z'}

# query_params = {"query": "from:OttawaPolice", "max_results": 100, "tweet.fields": "public_metrics"}
query_params = {"query": "from:business", "max_results": 100, "tweet.fields": "public_metrics"}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    json_response = connect_to_endpoint(search_url, query_params)
    
    tweets_df = pd.DataFrame(json_response['data'])
    #print(tweets_df[0:1])
    print(tweets_df[0:100]['text'])
    
    tweets_df.to_csv("tweets.csv", index=False)
    # tweets_df.info()
    # print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()
    
    


200
0     Brazilian poultry giant BRF is in talks to sel...
1     It’s unlikely that China's regional government...
2     Bath &amp; Body Works said Dan Loeb’s Third Po...
3     "For buyers brave enough to proceed despite 20...
4     Germany is pushing the European Commission to ...
                            ...                        
95    A Bloomberg investigation traces much of the a...
96    RT @BloombergLive: TOMORROW: How has @Delta re...
97    The iShares Short Treasury Bond ETF took in al...
98    RT @BloombergUK: "Significant progress has bee...
99    Low-cost carrier Wizz Air will suspend all fli...
Name: text, Length: 100, dtype: object


In [16]:
# Read the CSV file into a DataFrame
df = pd.read_csv('tweets.csv')

# View the first few rows of the DataFrame
#print(df['text'])

In [18]:
!pip install tensorflow
!pip install transformers
# Load the FinBERT model
from transformers import TFBertForSequenceClassification, BertTokenizer

model_name = 'ProsusAI/finbert'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
Pro_Positive = 0
Pro_Negative = 0
# Loop over the values in the "text" column and print the length of each one
for text in (df['text']):
    
  # Tokenize your financial texts
  text = text
  inputs = tokenizer(text, padding=True, truncation=True, return_tensors="tf")

  # Prepare your input
  input_ids = inputs["input_ids"]
  attention_mask = inputs["attention_mask"]

  # Run inference
  outputs = model(input_ids, attention_mask=attention_mask)
  scores = outputs[0][0].numpy()
  Pro_Positive += scores[1]
  Pro_Negative += scores[0]
  #print("Positive sentiment probability:", scores[1])
  #print("Negative sentiment probability:", scores[0])

print("Positive sentiment probability:", Pro_Positive)
print("Negative sentiment probability:", Pro_Negative)

Positive sentiment probability: 18.382268713787198
Negative sentiment probability: 27.545853207819164
